In [2]:
import dash
from dash import jupyter_dash
from dash import html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc
import dash_leaflet as dl
import requests
import pandas as pd
import plotly.express as px

# Example coordinates for Brussels, Belgium
example_lat, example_lon = 50.8503, 4.3517

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(fluid=True, children=[
    dbc.Row([
        dbc.Col(html.H1("AED Locator", className="text-center"), width=12)
    ]),
    dbc.Row([
        dbc.Col([
            dl.Map(center=(example_lat, example_lon), zoom=12, children=[
                dl.TileLayer(),
                dl.Marker(id='user-marker', position=(example_lat, example_lon), children=[
                    dl.Tooltip("Your Current Location"),
                    dl.Popup("Updating Location...")
                ]),
                dl.Marker(id='aed-marker', position=(0, 0), children=[
                    dl.Tooltip("Nearest AED Location"),
                    dl.Popup("Nearest AED")
                ])
            ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
        ], width=12)
    ]),
    dbc.Row([
        dbc.Col(dbc.Button("Get Directions to Nearest AED", color="primary", className="mt-3", id="get-directions-btn"), width=12)
    ]),
    dbc.Row([
        dbc.Col(html.Div(id="directions-output"), width=12)
    ])
])

# Placeholder callback for button
@callback(
    Output('user-marker', 'position'),
    Output('aed-marker', 'position'),
    Output('directions-output', 'children'),
    Input('get-directions-btn', 'n_clicks'),
    prevent_initial_call=True
)

def show_directions(n_clicks):
    if n_clicks:
        # Request user location from backend
        user_loc_response = requests.get("http://127.0.0.1:5000/get_location")
        user_loc = user_loc_response.json()

        # Request nearest AED from backend
        aed_response = requests.post("http://127.0.0.1:5000/nearest_aed", json={'location': user_loc})
        if aed_response.status_code == 200:
            nearest_aed = aed_response.json()
            return (user_loc['lat'], user_loc['lon']), (nearest_aed['lat'], nearest_aed['lon']), f"Directions to the nearest AED at {nearest_aed['address']}"
        else:
            return (user_loc['lat'], user_loc['lon']), (0, 0), "No AED found within radius"
    return (50.8503, 4.3517), (0, 0), ""

if __name__ == '__main__':
    app.run(jupyter_mode="external", port = 8051)

Dash app running on http://127.0.0.1:8051/


[2024-05-19 19:46:09,586] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/shyamkrishna/anaconda3/lib/python3.11/site-packages/requests/models.py", line 971, in json
    return complexjson.loads(self.text, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shyamkrishna/anaconda3/lib/python3.11/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shyamkrishna/anaconda3/lib/python3.11/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shyamkrishna/anaconda3/lib/python3.11/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Tra